# threads
That''s all the code you need to successfully create and instantiate a thread in python.

In [28]:
import threading
import time

class Worker(threading.Thread):
    # Our workers constructor, note the super() method which is vital if we want this
    # to function properly
    def __init__(self):
        super(Worker, self).__init__()

    def run(self):
        for i in range(10):
            print(i)
            time.sleep(0.01)
        
def main():
    # This initializes ''thread1'' as an instance of our Worker Thread
    thread1 = Worker()
    # This is the code needed to run our newly created thread
    thread1.start()

main()

0
1
2
3
4
5
6
7
8
9


In [29]:
thread1 = Worker()
thread2 = Worker()
thread3 = Worker()
thread1.start()
thread2.start()
thread3.start()

0
0
0
1
1
1
2
2
2
3
3
3
4
4
4
5
5
5
6
6
6
7
7
7
8
8
8
9
9
9


## Problems with global interpreter lock (GIL)

In [42]:
import threading
import time
import math

class FactorialWorker(threading.Thread):
    # Our workers constructor, note the super() method which is vital if we want this
    # to function properly
    def __init__(self):
        super(FactorialWorker, self).__init__()

    def run(self):
        res = math.factorial(1500000)
        
        
thread1 = FactorialWorker()
thread1.start()

In [43]:
thread1 = Worker()
thread2 = Worker()
thread3 = Worker()
thread1.start()
thread2.start()
thread3.start()

# multiprocessing
## Threads vs. Processes

In [44]:
import multiprocessing

def worker(num):
    """thread worker function"""
    print('Worker:', num)
    res = math.factorial(1500000)
    return

if __name__ == '__main__':
    jobs = []
    for i in range(5):
        p = multiprocessing.Process(target=worker, args=(i,))
        jobs.append(p)
        p.start()

Worker: 0
Worker: 1
Worker: 2
Worker: 3
Worker: 4


Each Process instance has a name with a default value that can be changed as the process is created. Naming processes is useful for keeping track of them, especially in applications with multiple types of processes running simultaneously.

In [5]:
import multiprocessing
import time

def worker():
    name = multiprocessing.current_process().name
    print(name, 'Starting')
    time.sleep(2)
    print(name, 'Exiting')

def my_service():
    name = multiprocessing.current_process().name
    print(name, 'Starting')
    time.sleep(3)
    print(name, 'Exiting')

if __name__ == '__main__':
    service = multiprocessing.Process(name='my_service', target=my_service)
    worker_1 = multiprocessing.Process(name='worker 1', target=worker)
    worker_2 = multiprocessing.Process(target=worker) # use default name

    worker_1.start()
    worker_2.start()
    service.start()

worker 1 Starting
Process-8 Starting
my_service Starting
worker 1 Exiting
Process-8 Exiting
my_service Exiting


## Daemon processes

## Waiting for processes
To wait until a process has completed its work and exited, use the `join()` method.

In [7]:
import os
 
from multiprocessing import Process
 
def doubler(number):
    """
    A doubling function that can be used by a process
    """
    result = number * 2
    proc = os.getpid()
    print('{0} doubled to {1} by process id: {2}'.format(
        number, result, proc))
 
if __name__ == '__main__':
    numbers = [5, 10, 15, 20, 25]
    procs = []
 
    for index, number in enumerate(numbers):
        proc = Process(target=doubler, args=(number,))
        procs.append(proc)
        proc.start()
 
    for proc in procs:
        proc.join()

5 doubled to 10 by process id: 22470
10 doubled to 20 by process id: 22473
15 doubled to 30 by process id: 22474
20 doubled to 40 by process id: 22477
25 doubled to 50 by process id: 22482


## Locks
The multiprocessing module supports locks in much the same way as the threading module does. All you need to do is import Lock, acquire it, do something and release it. Let’s take a look:

In [13]:
from multiprocessing import Process, Lock
 
def printer(item, lock):
    """
    Prints out the item that was passed in
    """
    print(item)
 
if __name__ == '__main__':
    lock = Lock()
    items = ['tangotangotangotangotangotangotango', 'foxtrotfoxtrotfoxtrotfoxtrotfoxtrot', 10]
    for item in items:
        p = Process(target=printer, args=(item, lock))
        p.start()

tangotangotangotangotangotangotango
foxtrotfoxtrotfoxtrotfoxtrotfoxtrot
10


In [9]:
from multiprocessing import Process, Lock
 
def printer(item, lock):
    """
    Prints out the item that was passed in
    """
    lock.acquire()
    try:
        print(item)
    finally:
        lock.release()
 
if __name__ == '__main__':
    lock = Lock()
    items = ['tango', 'foxtrot', 10]
    for item in items:
        p = Process(target=printer, args=(item, lock))
        p.start()

tango
foxtrot
10


## Pool class
The Pool class is used to represent a **pool of worker processes**. It has methods which can allow you to offload tasks to the worker processes. Let’s look at a really simple example:

In [15]:
from multiprocessing import Pool
 
def doubler(number):
    return number * 2
 
if __name__ == '__main__':
    numbers = [5, 10, 20]
    pool = Pool(processes=3)
    print(pool.map(doubler, numbers))

[10, 20, 40]


## Process Communication

In [17]:
from multiprocessing import Process, Queue
 
sentinel = -1
 
def creator(data, q):
    """
    Creates data to be consumed and waits for the consumer
    to finish processing
    """
    print('Creating data and putting it on the queue')
    for item in data:
 
        q.put(item)
 
 
def my_consumer(q):
    """
    Consumes some data and works on it
 
    In this case, all it does is double the input
    """
    while True:
        data = q.get()
        print('data found to be processed: {}'.format(data))
        processed = data * 2
        print(processed)
 
        if data is sentinel:
            break
 
 
if __name__ == '__main__':
    q = Queue()
    data = [5, 10, 13, -1]
    process_one = Process(target=creator, args=(data, q))
    process_two = Process(target=my_consumer, args=(q,))
    process_one.start()
    process_two.start()
 
    q.close()
    q.join_thread()
 
    process_one.join()
    process_two.join()

Creating data and putting it on the queue
data found to be processed: 5
10
data found to be processed: 10
20
data found to be processed: 13
26
data found to be processed: -1
-2


## Wrapping Up
We have a lot of material here. 
You have learned how to use the multiprocessing module to target regular functions, communicate between processes using Queues, naming threads and much more. There is also a lot more in the Python documentation that isn’t even touched in this article, so be sure to dive into that as well. In the meantime, you now know how to utilize all your computer’s processing power with Python!